In [ ]:
import pandas as pd
import numpy as np

# Constants
latitude_deg = 27.7  # Example latitude in degrees (change this to your location!)

# Read your Excel file
df = pd.read_excel(r"C:\Users\DELL\Downloads\PETDATA.xlsx")  # <-- change 'your_file.xlsx' to your real filename

# Prepare extra column for PET
PET_list = []

# Function to calculate Ra (extraterrestrial radiation)
def calculate_ra(julian_day, latitude_deg):
    doy_factor = 2 * np.pi / 365
    dr = 1 + 0.033 * np.cos(doy_factor * julian_day)
    solar_declination = 0.409 * np.sin(doy_factor * julian_day - 1.405)
    latitude_rad = np.radians(latitude_deg)
    sunset_hour_angle = np.arccos(-np.tan(latitude_rad) * np.tan(solar_declination))
    Ra = (24 * 60 / np.pi) * 0.082 * dr * (
        sunset_hour_angle * np.sin(latitude_rad) * np.sin(solar_declination) +
        np.cos(latitude_rad) * np.cos(solar_declination) * np.sin(sunset_hour_angle)
    )
    return Ra  # in MJ/m2/day

# Assign approximate Julian day for each month
julian_days = [15, 45, 75, 105, 135, 162, 198, 228, 258, 288, 318, 344]  # Middle of each month

# Loop through each row
for i, row in df.iterrows():
    Tmax = row['Tmax']
    Tmin = row['Tmin']
    Temperature = (Tmax + Tmin) / 2
    julian_day = julian_days[i]  # i = month index
    
    Ra = calculate_ra(julian_day, latitude_deg)
    
    PET = max(0.0023 * Ra * 0.408 * ((Tmax - Tmin) ** 0.5) * (Temperature + 17.8), 0)
    PET_list.append(PET)

# Add PET to DataFrame
df['PET (mm/day)'] = PET_list

# Save to new Excel file
df.to_excel('PET_results.xlsx', index=False)

print("PET calculated and saved to 'PET_results.xlsx'!")
